In [ ]:
import gymGharrafa

In [ ]:
import gym

In [ ]:
from tensorboardX import SummaryWriter
import ptan
import common

import torch
import torch.nn as nn
import torch.nn.utils as nn_utils
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

In [ ]:
GAMMA = 0.99
LEARNING_RATE = 0.00001
ENTROPY_BETA = 0.001
NUM_ENVS = 1

REWARD_STEPS = 10
CLIP_GRAD = 0.9

In [ ]:
class NetPG(nn.Module):
    def __init__(self, input_size, n_actions):
        super(NetPG, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
name = "Train_PG_timecritic_entropy001_Gt_mobavg_noclip_00001lr"
device = torch.device('cuda:0')

In [ ]:
env = gym.make("gymGharrafa-v0")
writer = SummaryWriter(comment="Gharrafa_" + name)
net = NetPG(env.observation_space.shape[1], env.action_space.n).to(device)
print(net)

optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [ ]:
n_actions = 11
env.reset()
obs = env.reset()
#get first state
obsTensor = torch.FloatTensor(np.expand_dims(obs, axis=0)).to(device)

In [ ]:
nsteps = 10
meanG=0
done_episodes = 0
nreward_sum = 0
return_sum = 0
idx = 0
baseline = 0
action_set = range(n_actions)
GtAcc = np.zeros(500)
vGt = np.zeros(500)

while meanG < 3000:
    episode_over = False
    timestep = 0
    step_rewards = []
    step_states = []
    step_actions = []
    while(not episode_over):
        #get action
        logits_v = net(obsTensor)[0]
        softmax = F.softmax(logits_v)
        
        step_states.append(obsTensor)
        
        #select the action randomly following distribution from softmax
        action = np.random.choice(action_set,p=softmax.detach().cpu().numpy())
        obs,reward,episode_over,additional = env.step(action)
        step_actions.append(action)
        obsTensor = torch.FloatTensor(np.expand_dims(obs, axis=0)).to(device)
        
        step_rewards.append(reward)
        
    done_episodes += 1
    
    G = np.sum(step_rewards)
    if G < 20:
        print("Check SUMO for errors: reward too low.")
        break
        
    return_sum += G
    meanG = return_sum/done_episodes
    
    writer.add_scalar("return_mean", meanG, done_episodes)
    writer.add_scalar("return", G, done_episodes)
    writer.add_scalar("episode_length", len(step_rewards), done_episodes)
    
    
    accR = 0
    
    for t,R in enumerate(step_rewards):
        idx += 1
        #batch_delta =
        
        Gt = np.sum(step_rewards[t:])
        GtAcc[t] += Gt
        
        
        
        delta = torch.FloatTensor([Gt-vGt[t]]).to(device)
        

        if vGt[t] == 0 :
            vGt[t] = Gt
        else:
            vGt[t] = 0.5*vGt[t] + 0.5*Gt
        
        writer.add_scalar("baseline", vGt[0], done_episodes)
            
        
        optimizer.zero_grad()
        
        
        logits_v = net(step_states[t])
        
        log_prob_v = F.log_softmax(logits_v)
        
        log_prob_actions_v = delta * log_prob_v[0,step_actions[t]]
        loss_policy_v = -log_prob_actions_v.mean()
        
        prob_v = F.softmax(logits_v, dim=1)
        entropy_v = -(prob_v * log_prob_v).sum(dim=1).mean()
        entropy_loss_v = -ENTROPY_BETA * entropy_v
        
        loss_v = loss_policy_v + entropy_loss_v

        loss_v.backward()

        optimizer.step()
    
    print("Episode %d ended with return %2f. Average is %2f. Last timestep %2f." % (done_episodes,G,meanG,additional["time"]))
    